# 6.1 이미지 분류를 위한 신경망
## 목차
### 6.1.1 LeNet-5
### 6.1.2 AlexNet
### 6.1.3 VGGNet
### 6.1.4 GoogLeNet
### 6.1.5 ResNet

## 6.1 LeNet-5
- convolution, sub-sampling을 반복적으로 거치면서 마지막 fully connected layer에서 분류 수행